# Triple check vis
- Deconstruct vis class and check it piece by piece using older data

In [ ]:
import pandas as pd

### *read_eval_from_file()*
```python
def read_eval_from_file(self, s_item_csv, g_item_csv):
    self.strain_i_hist = pd.read_csv(self.model_folder + '/' + s_item_csv)
    self.grain_i_hist = pd.read_csv(self.model_folder + '/' + g_item_csv)
```

In [ ]:
strain_i_hist = pd.read_csv('Strain.Jason.csv')
grain_i_hist = pd.read_csv('Grain.Jason.csv')

### *parse_strain_cond_df()*
```python
def parse_strain_cond_df(self, cond):
    self.scdf = self.strain_i_hist[['code_name', 'epoch', 'sample_mil', 'timestep',
                                    'unit_time', cond, 'input_s', 'acc', 'sse']]
    self.scdf = self.scdf.groupby(['code_name', 'epoch', 'timestep', cond],
                                  as_index=False).mean() 
    self.scdf['cond'] = self.scdf[cond]
    self.scdf['exp'] = 'strain'
```

Look at the raw data file structure

In [ ]:
print(strain_i_hist.columns)
strain_i_hist.pivot_table(index='ID')

In [ ]:
# Easier label for ID
id_label = {61692410: 'Surface dys (Low Hidden)',
            62120867: 'Phonological dys (High p noise = 6!)',
            64195158: 'Typical developing'}

# Select useful columns only
scdf = strain_i_hist.loc[:,['ID', 'Trial.Scaled', 'Consistency', 'Freq', 'Accuracy', 'SSE']]
scdf['group'] = scdf.ID.map(id_label)
scdf['cond'] = scdf.Consistency + '_' + scdf.Freq
scdf = scdf.groupby(['group', 'Trial.Scaled', 'cond'], as_index=False).mean() 
scdf.rename(columns={'Trial.Scaled':'epoch',        # Trial scaled renamed to Epoch onward
                     'group': 'code_name',          # group renamed to code_name onward
                     'Accuracy': 'acc',
                     'SSE': 'sse'}, inplace=True)   
scdf['exp'] = 'strain'
scdf

### *parse_grain_cond_df()*

```python
def parse_grain_cond_df(self, cond):
    self.gcdf = self.grain_i_hist[['code_name', 'epoch', 'sample_mil', 'timestep',
                                   'unit_time', cond, 'input_s',
                                   'acc_acceptable', 'sse_acceptable',
                                   'acc_small_grain', 'sse_small_grain',
                                   'acc_large_grain', 'sse_large_grain']]
    self.gcdf = self.gcdf.rename(columns={'acc_acceptable':'acc', 'sse_acceptable':'sse'})
    self.gcdf = self.gcdf.groupby(['code_name', 'epoch', 'timestep', cond],
                                  as_index=False).mean()
    self.gcdf['cond'] = self.gcdf[cond]
    self.gcdf['exp'] = 'grain'
```

In [ ]:
print(grain_i_hist.columns)
grain_i_hist.pivot_table(index='ID')

ID is the same as Strain, will re-use id_label

In [ ]:
gcdf = grain_i_hist.loc[:,['ID', 'Trial.Scaled', 'NWType', 'Acceptable_hit', 'SSE']]
gcdf['group'] = gcdf.ID.map(id_label)
gcdf.rename(columns={'NWType':'cond'}, inplace=True)
gcdf = gcdf.groupby(['group', 'Trial.Scaled', 'cond'], as_index=False).mean() 
gcdf['exp'] = 'grain'
gcdf.rename(columns={'Acceptable_hit':'acc',
                     'Trial.Scaled':'epoch',        # Trial scaled renamed to Epoch onward
                     'group': 'code_name',
                     'SSE': 'sse'}, inplace=True)   
gcdf

### *parse_cond_df()*

```python
def parse_cond_df(self, cond_strain='condition_pf', cond_grain='condition', output=None):
    self.parse_strain_cond_df(cond_strain)
    self.parse_grain_cond_df(cond_grain)
    self.cdf = pd.concat([self.scdf, self.gcdf], sort=False)

    if output is not None:
        self.cdf.to_csv(output, index=False)
        print('Saved file to {}'.format(output))
```

In [ ]:
cdf = pd.concat([scdf, gcdf], sort=False)
cdf

### *make_df_wnw()*

``` python
def make_df_wnw(df, selected_cond):
    """
    This function make a word vs. nonword data file for plotting
    1) filter to last time step
    2) filter by selected_cond
    3) pivot by experiment (exp) and clean

    Inputs: 
        df: compiled batch results data file (cond is the filter column)
        selected_cond: select the condition in word and nonword condition 

    Output:
        plt_df: datafile for plotting with these columns:
            code_name, epoch, nonword_acc, word_acc
    """

    df_sel = df.loc[(df.timestep == df.timestep.max()) &
                    (df.cond.isin(selected_cond)),
                    ['code_name', 'epoch', 'acc', 'exp']]

    pvt = df_sel.pivot_table(index=['code_name', 'epoch'],
                             columns='exp').reset_index()

    plt_df = pd.DataFrame()
    plt_df['code_name'] = pvt.code_name
    plt_df['epoch'] = pvt.epoch
    plt_df['nonword_acc'] = pvt.acc.grain
    plt_df['word_acc'] = pvt.acc.strain

    return plt_df
```

In [ ]:
cdf.cond.unique()

In [ ]:
df_sel = cdf.loc[cdf.cond.isin(['INC_HF', 'NW_AMB', 'NW_UN'])]

pvt = df_sel.pivot_table(index=['code_name', 'epoch'],
                             columns='exp').reset_index()

pvt.sample(5)

In [ ]:
wnw_df = pd.DataFrame()
wnw_df['code_name'] = pvt.code_name        # Code name is "group label" onward
wnw_df['epoch'] = pvt.epoch     
wnw_df['nonword_acc'] = pvt.acc.grain
wnw_df['word_acc'] = pvt.acc.strain
wnw_df.sample(5)

### *plot_wnw()*


``` python
def plot_wnw(self, selected_cond):

    wnw_df = make_df_wnw(self.cdf, selected_cond=['INC_HF', 'ambiguous', 'unambiguous'])

    wnw_plot = (
        alt.Chart(wnw_df).mark_line(point=True).encode(
            y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
            x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
            color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen")),
            tooltip=["code_name", "word_acc", "nonword_acc"],
        ).properties(
            title="Word vs. Nonword accuracy at final time step"
        )
    )

    # Plot diagonal
    diagline = alt.Chart(pd.DataFrame({
        'x': [0, 1],
        'y': [0, 1]
    })).mark_line().encode(x=alt.X('x', axis=alt.Axis(labels=False)), 
                           y=alt.Y('y', axis=alt.Axis(labels=False)))

    wnw_with_diag = diagline + wnw_plot

    return wnw_with_diag
```


In [ ]:
import altair as alt

wnw_plot = (
        alt.Chart(wnw_df).mark_line(point=True).encode(
            y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
            x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
            color="code_name",
            tooltip=["code_name", "word_acc", "nonword_acc", 'epoch'],
        ).properties(
            title="Word vs. Nonword accuracy at final time step"
        )
    )

wnw_plot

Same as what reported in Rueckl, 2019
- High noise do yield relatively weaker nonword reading
- Low hidden learn slower but with a similar tranjectory shape to TD

### Suggestions

- Check the script for generating raw data...

We have this pipeline:

1. Data_cleaning
2. Modeling
3. Evaluator <-- I did manually check it during dev... haven't change much, but good to cross check with your script?
4. Vis (Parse raw data generated from evaluator) <-- we just checked this (and I do the checking manually in excel with a few cells before), it looks fine.

# Vis class source code

In [ ]:
class vis():
    # Visualize single model
    # Which will parse item level data to condition level data
    # Then plot with Altair
    def __init__(self):
        from evaluate import training_history, plot_development
        import altair as alt
                          
        self.read_eval_from_file(s_item_csv, g_item_csv)
        self.max_epoch = self.strain_i_hist['epoch'].max()

    def load_config(self):
        from meta import model_cfg
        self.cfg = model_cfg(self.model_folder + '/model_config.json', bypass_chk=True)
        
    def training_hist(self):
        self.t_hist = training_history(self.cfg.path_history_pickle)
        return self.t_hist.plot_all()
        
    def read_eval_from_file(self, s_item_csv, g_item_csv):
        self.strain_i_hist = pd.read_csv(self.model_folder + '/' + s_item_csv)
        self.grain_i_hist = pd.read_csv(self.model_folder + '/' + g_item_csv)
    
    # Condition level parsing
    def parse_strain_cond_df(self, cond):
        self.scdf = self.strain_i_hist[['code_name', 'epoch', 'sample_mil', 'timestep',
                                        'unit_time', cond, 'input_s', 'acc', 'sse']]
        self.scdf = self.scdf.groupby(['code_name', 'epoch', 'timestep', cond],
                                      as_index=False).mean() 
        self.scdf['cond'] = self.scdf[cond]
        self.scdf['exp'] = 'strain'
        
    def parse_grain_cond_df(self, cond):
        self.gcdf = self.grain_i_hist[['code_name', 'epoch', 'sample_mil', 'timestep',
                                       'unit_time', cond, 'input_s',
                                       'acc_acceptable', 'sse_acceptable',
                                       'acc_small_grain', 'sse_small_grain',
                                       'acc_large_grain', 'sse_large_grain']]
        self.gcdf = self.gcdf.rename(columns={'acc_acceptable':'acc', 'sse_acceptable':'sse'})
        self.gcdf = self.gcdf.groupby(['code_name', 'epoch', 'timestep', cond],
                                      as_index=False).mean()
        self.gcdf['cond'] = self.gcdf[cond]
        self.gcdf['exp'] = 'grain'
        
    def parse_cond_df(self, cond_strain='condition_pf', cond_grain='condition', output=None):
        self.parse_strain_cond_df(cond_strain)
        self.parse_grain_cond_df(cond_grain)
        self.cdf = pd.concat([self.scdf, self.gcdf], sort=False)

        if output is not None:
            self.cdf.to_csv(output, index=False)
            print('Saved file to {}'.format(output))
        
    # Visualization
    def plot_dev(self, y, exp=None, condition='cond', timestep=None):
        
        if timestep == None: timestep=self.cfg.n_timesteps
        timestep -= 1 # Reindex

        # Select data
        if exp is not None: 
            plot_df = self.cdf.loc[(self.cdf.exp==exp) & (self.cdf.timestep==timestep),]
        else:
            plot_df = self.cdf.loc[self.cdf.timestep==timestep,]

        # Plotting
        title = '{} at timestep {} / unit time {}'.format(y, timestep + 1, self.cfg.max_unit_time)
        sel = alt.selection(type='single', on='click', fields=[condition], empty='all')
        plot = alt.Chart(
                    plot_df
                ).mark_line(
                    point=True
                ).encode(
                    y=alt.Y(y, scale=alt.Scale(domain=(0, 1))),
                    x='epoch:Q',
                    color=condition,
                    opacity=alt.condition(sel, alt.value(1), alt.value(0)),
                    tooltip=['epoch', 'timestep', 'sample_mil', 'acc', 'sse']
                ).add_selection(sel
                ).interactive(
                ).properties(title=title)

        return plot
    
    def plot_dev_interactive(self, y, exp=None, condition='cond'):
        
        # Condition highlighter from legend
        select_cond = alt.selection(
            type='multi', on='click', fields=[condition], empty='all', bind="legend"
        )
        
        # Slider timestep filter
        slider_time = alt.binding_range(min=0, max=self.cfg.n_timesteps - 1, step=1)
        select_time = alt.selection_single(
            name="filter",
            fields=['timestep'],
            bind=slider_time,
            init={'timestep': self.cfg.n_timesteps - 1}
        )
        
        # Interactive development plot
        plot_dev = alt.Chart(self.cdf).mark_line(point=True).encode(
            y=alt.Y(y, scale=alt.Scale(domain=(0, 1))),
            x='epoch:Q',
            color=condition,
            opacity=alt.condition(select_cond, alt.value(1), alt.value(0.1)),
            tooltip=['epoch', 'timestep', 'sample_mil', 'acc', 'sse']
        ).add_selection(select_time, select_cond).transform_filter(select_time).properties(
            title='Development plot'
        )

        return plot_dev

    def plot_time(self, y, exp=None, condition='cond', epoch=None):  
        if epoch == None: epoch = self.max_epoch

        # Select data
        if exp is not None: 
            plot_df = self.cdf.loc[(self.cdf.exp==exp) & (self.cdf.epoch == epoch),]
        else:
            plot_df = self.cdf.loc[self.cdf.epoch == epoch,]

        # Plotting
        title = '{} at epoch {} '.format(y, epoch)
        sel = alt.selection(type='single', on='click', fields=[condition], empty='all')
        
        plot = alt.Chart(
                    plot_df
                ).mark_line(
                    point=True
                ).encode(
                    y=alt.Y(y, scale=alt.Scale(domain=(0, 1))),
                    x='unit_time:Q',
                    color=condition,
                    opacity=alt.condition(sel, alt.value(1), alt.value(0)),
                    tooltip=['epoch', 'timestep', 'sample_mil', 'acc', 'sse']
                ).add_selection(sel
                ).interactive(
                ).properties(title=title)

        return plot
    
    def plot_time_interactive(self, y, exp=None, condition='cond'):
        
        # Condition highlighter from legend
        select_cond = alt.selection(
            type='multi', on='click', fields=[condition], empty='all', bind="legend"
        )
            
        # Slider epoch filter
        slider_epoch = alt.binding_range(
            min=self.cfg.save_freq, max=self.cfg.nEpo, step=self.cfg.save_freq
        )
        
        select_epoch = alt.selection_single(
            name="filter",
            fields=['epoch'],
            bind=slider_epoch,
            init={'epoch': self.cfg.nEpo}
        )
        
        # Plot
        plot_time = alt.Chart(self.cdf).mark_line(point=True).encode(
            y=alt.Y(y, scale=alt.Scale(domain=(0, 1))),
            x='unit_time:Q',
            color=condition,
            opacity=alt.condition(select_cond, alt.value(1), alt.value(0.1)),
            tooltip=['epoch', 'timestep', 'sample_mil', 'acc', 'sse']
        ).add_selection(select_epoch, select_cond).transform_filter(select_epoch).properties(
            title='Interactive time plot',
        )
        
        return plot_time
    
    def plot_wnw(self, selected_cond):
    
        wnw_df = make_df_wnw(self.cdf, selected_cond=selected_cond)

        wnw_plot = (
            alt.Chart(wnw_df).mark_line(point=True).encode(
                y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
                x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
                color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen")),
                tooltip=["code_name", "word_acc", "nonword_acc"],
            ).properties(
                title="Word vs. Nonword accuracy at final time step"
            )
        )
        
        # Plot diagonal
        diagline = alt.Chart(pd.DataFrame({
            'x': [0, 1],
            'y': [0, 1]
        })).mark_line().encode(x=alt.X('x', axis=alt.Axis(labels=False)), 
                               y=alt.Y('y', axis=alt.Axis(labels=False)))

        wnw_with_diag = diagline + wnw_plot
        
        return wnw_with_diag
    
    def plots(self, mode, ys, cond_strain='condition_pf', cond_grain='condition'):
        # Mode = dev(d) / time(t)
        self.parse_cond_df(cond_strain, cond_grain)
        
        plots = alt.hconcat()
        for y in ys:
            if mode == 'd':
                plots |= self.plot_dev(y)
            elif mode == 't':
                plots |= self.plot_time(y, self.max_epoch)
            else:
                print('Use d for development plot, use t for time plot')
            
        return plots
        
    def export_result(self):
        self.parse_cond_df()
        return self.cdf.reset_index(drop=True)